In [1]:
#evolutionary game
import pygame
import random
import numpy as np

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# constants
inter_radius = 1
offspring_radius = 1
death_rate = 0.1

# colors
col_about_to_die = (200, 200, 225)
col_type1 = (255, 255, 0)
col_type2 = (255, 0, 255)
col_background = (10, 10, 40)
col_grid = (30, 30, 60)

In [3]:
# helper functions
def radius_rand(radius, r, c, cur, value):
    chosen_pos = None
    chosen_value = None
    
    # defining the begin and end of the neighberhood
    beginR = max((r-radius),0)
    endR = min((r+radius+1),(cur.shape[0]-1)) 
    beginC = max((c-radius), 0)
    endC = min((c+radius+1), (cur.shape[1]-1))

    #print(beginR, endR, beginC, endC)
    #    return chosen_pos, chosen_value
    
    for i in range(8*radius):
        rand_r = random.randint(beginR, endR)
        rand_c = random.randint(beginC, endC)
        if value == 0:
            if cur[rand_r, rand_c] == 0:
                chosen_pos = [rand_r, rand_c]
                chosen_value = 0
                return chosen_pos, chosen_value
        else: 
            if cur[rand_r, rand_c] != 0:
                chosen_pos = [rand_r, rand_c]
                chosen_value = cur[rand_r, rand_c]
        
                return chosen_pos, chosen_value
            
    return chosen_pos, chosen_value

def offspring_fitness(vocal, mate):
    #place_matrix = np.zeros((2,2))
    #A = np.array([[1,2],
    #          [2,1]])
        
    if vocal == 1 and mate == 1:
        nr_offspring = 1
    elif vocal == 1 and mate== 2 :
        nr_offspring = 2
    elif vocal == 2 and mate == 1 :
        nr_offspring = 2
    else:
        nr_offspring = 1
        
    #nr_offspring = place_matrix.dot(A)
    return nr_offspring

def cemetery(cur):
    nxt = np.zeros((cur.shape[0], cur.shape[1]))
    for r, c in np.ndindex(cur.shape):
        nxt[r,c] = cur[r,c]
        
        death = np.random.uniform(0.0, 1.0, 1)
        
        if death <= death_rate:
            nxt[r,c]= 0 
    return nxt
        

In [4]:
# EA

def mating_pool(cur):
    nxt = np.zeros((cur.shape[0], cur.shape[1]))
    for r, c in np.ndindex(cur.shape):
        vocal = cur[r,c]
        nxt[r,c] = vocal
        # lets kill some of the old generation
        death = np.random.uniform(0.0, 1.0, 1)
        
        if death <= death_rate:
            nxt[r,c]= 0  
        # pick a random individual from inetarction_radius to mate
        mate_pos,mate_value = radius_rand(inter_radius, r, c, cur, 1)
        
        if vocal >2 or vocal<0:
            raise Exception("The type of the player is unknown")
            
        # Placing the offspring in offspring_radius according to A = [[1,2],[2,1]]
        if mate_value is not None:

            nr_offspring = offspring_fitness(vocal, mate_value)
            #print("offspring", nr_offspring)
            for i in range(nr_offspring):
                offspring_pos, offspring_value = radius_rand(offspring_radius, r, c, cur, 0)
                #print("offspring_pos", offspring_pos, offspring_value)
          
                if offspring_value is not None:

                    nxt[offspring_pos[0], offspring_pos[1]] = vocal
        #else:
         #   raise Exception("None type in offspring")
                
    return nxt

    
    

In [5]:
def update(surface, cur, sz):
    

    nxt = mating_pool(cur)
     
    for r, c in np.ndindex(nxt.shape):
        if nxt[r, c] == 1:
            col = col_type1
        elif nxt[r, c] == 2:
            col = col_type2
        else:
            col = col_background
            
        pygame.draw.rect(surface, col, (c*sz, r*sz, sz-1, sz-1))
        
    return nxt

def init(dimx, dimy):
    cells = np.zeros((dimy, dimx))
    pattern = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0],
                        [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0],
                        [1,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [2,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]);
    pos = (50,50)
    cells[pos[0]:pos[0]+pattern.shape[0], pos[1]:pos[1]+pattern.shape[1]] = pattern
    return cells


In [6]:
# main
def main(dimx, dimy, cellsize):
    pygame.init()
    surface = pygame.display.set_mode((dimx * cellsize, dimy * cellsize))
    pygame.display.set_caption("Evolutionary Game")
    generation = 0
    cells = init(dimx, dimy)
    finalMatrix = cells.copy()
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return finalMatrix
        generation = generation+1
        surface.fill(col_grid)

        cells = update(surface, cells, cellsize)
        finalMatrix = np.dstack((finalMatrix, cells))
        pygame.display.update()
        if generation == 100:
            pygame.quit()
            return finalMatrix

In [7]:
# call main
if __name__ == '__main__':
    finalMatrix = main(120,90,8)

In [8]:
file = open("evoOutputMatrix.txt", "w")
for row in finalMatrix:
    np.savetxt(file, row)
file.close()

In [9]:
original = np.loadtxt("evoOutputMatrix.txt").reshape(finalMatrix.shape)
compare = original == finalMatrix
print(compare.all())

True
